<h1>Groceries recommendation engine</h1>

In [19]:
#Importing required libraries
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from textblob import Word
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from rake_nltk import Rake
import pickle

In [2]:
#reading the input files
df = pd.read_json(r"C:\Users\Santhosh Kumar\Downloads\meta_Grocery_and_Gourmet_Food.json\meta_Grocery_and_Gourmet_Food.json", lines=True)
df.head()

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,"[Grocery & Gourmet Food, Dairy, Cheese & Eggs,...",,"[BEEMSTER GOUDA CHEESE AGED 18/24 MONTHS, Stat...",,Beemster Gouda - Aged 18/24 Months - App. 1.5 Lbs,[],[],,Ariola Imports,[],"165,181 in Grocery & Gourmet Food (","[B0000D9MYM, B0000D9MYL, B00ADHIGBA, B00H9OX59...",Grocery,,,$41.91,0681727810,NaN
1,"[Grocery & Gourmet Food, Cooking & Baking, Sug...",,"[Shipped from UK, please allow 10 to 21 busine...",,Trim Healthy Mama Xylitol,"[B01898YHXK, B01BCM6LAC, B00Q4OL47O, B00Q4OL5Q...",[https://images-na.ssl-images-amazon.com/image...,,,[],"315,867 in Grocery & Gourmet Food (",[],Grocery,,,,0853347867,NaN
2,"[Grocery & Gourmet Food, Cooking & Baking, Fro...",,[Jazz up your cakes with a sparkling monogram ...,,Letter C - Swarovski Crystal Monogram Wedding ...,[],[],,Unik Occasions,[],"[>#669,941 in Kitchen & Dining (See Top 100 in...",[B07DXN65TF],Amazon Home,,"September 21, 2010",$29.95,1888861118,NaN
3,"[Grocery & Gourmet Food, Cooking & Baking, Fro...",,"[Large Letter - Height 4.75""]",,Letter H - Swarovski Crystal Monogram Wedding ...,[],[],,Other,"[Large Letter - Height 4.75""]","[>#832,581 in Kitchen & Dining (See Top 100 in...",[],Amazon Home,,"September 11, 2011",$11.45,1888861517,NaN
4,"[Grocery & Gourmet Food, Cooking & Baking, Fro...",,"[4.75""]",,Letter S - Swarovski Crystal Monogram Wedding ...,[],[],,Unik Occasions,"[4.75"" height]","[>#590,999 in Kitchen & Dining (See Top 100 in...",[],Amazon Home,,"September 11, 2011",$15.00,1888861614,NaN


In [3]:
#Selecting only the required columns for further analysis
df_v1 = df[['title','description','brand']]
df_v1.head()

,title,description,brand
0,Beemster Gouda - Aged 18/24 Months - App. 1.5 Lbs,"[BEEMSTER GOUDA CHEESE AGED 18/24 MONTHS, Stat...",Ariola Imports
1,Trim Healthy Mama Xylitol,"[Shipped from UK, please allow 10 to 21 busine...",
2,Letter C - Swarovski Crystal Monogram Wedding ...,[Jazz up your cakes with a sparkling monogram ...,Unik Occasions
3,Letter H - Swarovski Crystal Monogram Wedding ...,"[Large Letter - Height 4.75""]",Other
4,Letter S - Swarovski Crystal Monogram Wedding ...,"[4.75""]",Unik Occasions


In [4]:
len(df_v1)

287051

In [5]:
#Selecting only top 5000 records for further analysis
df_v1 = df_v1.head(5000)

In [6]:
#Business logic to remove noises from the dataset
def pre_processing(df, stop_words):
    df['description'] = df['description'].apply(lambda x: " ".join(x.lower() for x in x))
    df['description'] = df['description'].str.replace('[^\w\s]', '')
    df['description'] = df['description'].str.replace('\d', '')
    df['description'] = df['description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words)) 
    df['description'] = df['description'].apply(lambda x: " ".join([Word(x).lemmatize() for x in x.split()]))
    return df

In [7]:
#Preprocessing the data
stop_words = stopwords.words('english')
train = pre_processing(df_v1,stop_words)
train.head()

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


,title,description,brand
0,Beemster Gouda - Aged 18/24 Months - App. 1.5 Lbs,beemster gouda cheese aged month statement reg...,Ariola Imports
1,Trim Healthy Mama Xylitol,shipped uk please allow business day arrival e...,
2,Letter C - Swarovski Crystal Monogram Wedding ...,jazz cake sparkling monogram sparkling collect...,Unik Occasions
3,Letter H - Swarovski Crystal Monogram Wedding ...,large letter height,Other
4,Letter S - Swarovski Crystal Monogram Wedding ...,,Unik Occasions


In [9]:
#Concatinating description and brand columns
train['description'] = train['description']+" "+train['brand']
train.head()

,title,description,brand
0,Beemster Gouda - Aged 18/24 Months - App. 1.5 Lbs,beemster gouda cheese aged month statement reg...,Ariola Imports
1,Trim Healthy Mama Xylitol,shipped uk please allow business day arrival e...,
2,Letter C - Swarovski Crystal Monogram Wedding ...,jazz cake sparkling monogram sparkling collect...,Unik Occasions
3,Letter H - Swarovski Crystal Monogram Wedding ...,large letter height Other,Other
4,Letter S - Swarovski Crystal Monogram Wedding ...,Unik Occasions,Unik Occasions


In [10]:
#Dropping the column brand
train_v1 = train.drop(['brand'],axis=1)

In [12]:
#Checking for null values
train_v1.isnull().sum()

title          0
description    0
dtype: int64

In [14]:
#Replacing blank spaces with null
train_v1 = train_v1.replace(r'^\s*$', np.nan, regex=True)

In [15]:
#Checking if there is any null values
train_v1.isnull().sum()

title            0
description    211
dtype: int64

In [16]:
#Dropping null values
train_v1=train_v1.dropna()
train_v1.head()

,title,description
0,Beemster Gouda - Aged 18/24 Months - App. 1.5 Lbs,beemster gouda cheese aged month statement reg...
1,Trim Healthy Mama Xylitol,shipped uk please allow business day arrival e...
2,Letter C - Swarovski Crystal Monogram Wedding ...,jazz cake sparkling monogram sparkling collect...
3,Letter H - Swarovski Crystal Monogram Wedding ...,large letter height Other
4,Letter S - Swarovski Crystal Monogram Wedding ...,Unik Occasions


In [18]:
#Applying count vectorization
count = CountVectorizer()
count_matrix = count.fit_transform(train_v1['description'])

In [22]:
#Finding cosine similarity
cosine_sim = cosine_similarity(count_matrix)

In [23]:
#SToring the cosine similarity matrix as pickle file for later purpose to build flask application
pickle.dump(cosine_sim, open(r"cosine_similarity.pkl", "wb"))

In [25]:
cosine_pickle = pickle.load(open(r"cosine_similarity.pkl", "rb"))

In [26]:
#SToring the dataframe as pickle file for later purpose to build flask application
train_v1.to_pickle("recommendation_data.pkl")

In [27]:
df = pd.read_pickle("recommendation_data.pkl")
df.set_index('title', inplace = True)

In [29]:
#Converting the index to series
indices = pd.Series(df.index)
indices[:5]

0    Beemster Gouda - Aged 18/24 Months - App. 1.5 Lbs
1                            Trim Healthy Mama Xylitol
2    Letter C - Swarovski Crystal Monogram Wedding ...
3    Letter H - Swarovski Crystal Monogram Wedding ...
4    Letter S - Swarovski Crystal Monogram Wedding ...
Name: title, dtype: object

In [32]:
# defining the function that takes in product title 
# As input and returns the top 5 recommended products
def recommendations(title, cosine_sim = cosine_pickle):
    
    # initializing the empty list of recommended products
    recommended_items = []
    # gettin the index of the product that matches the title
    idx = indices[indices == title].index[0]
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    # getting the indexes of the 10 most similar products
    top_10_indexes = list(score_series.iloc[1:6].index)
    # populating the list with the titles of the best 10 matching products
    for i in top_10_indexes:
        recommended_items.append(list(df.index)[i])
        
    return recommended_items

In [33]:
#Calling the recommendation function to return top 5 similar items
recommendations('Letter C - Swarovski Crystal Monogram Wedding Cake Topper Letter')

["My Grandma's of New England Coffee Cake Custom Sampler",
 "Sweet'N Low&reg; Bakery Mixes Assortment - 12 pack",
 "Sweet'N Low&reg; Bakery Mixes Assortment - 12 pack",
 'Schlunder Liqueur Cake - Black Forest (14 ounce)',
 'Schlunder Liqueur Cakes - Jamaican Rum (14 ounce)']